In [1]:
import torch
from tqdm import tqdm
import numpy as np
import gc
# from sklearn.svm import LinearSVC
from cuml.svm import LinearSVC
import numpy as np

from transformers import AutoTokenizer, AutoModelForCausalLM
from emb_vectors_functions import find_self_embeds, get_shadow_ratios
from model_loading import get_weight_by_name

In [2]:
# "meta-llama/Llama-3.1-70B", "meta-llama/Llama-3.1-8B"
# "Qwen/Qwen2.5-0.5B-Instruct", "Qwen/Qwen2.5-1.5B-Instruct", "Qwen/Qwen2.5-3B-Instruct", "Qwen/Qwen2.5-7B-Instruct"
# "Qwen/Qwen2.5-14B-Instruct", "Qwen/Qwen2.5-32B-Instruct", "Qwen/Qwen2.5-72B-Instruct"
model_name = "meta-llama/Llama-3.1-70B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
embeddings = get_weight_by_name(model_name, "head")
# embeddings = embeddings.cuda()
embeddings = embeddings.float()
# embeddings = torch.randn_like(embeddings)
embeddings.requires_grad = False

Fetching 50 files:   0%|          | 0/50 [00:00<?, ?it/s]

In [3]:
fail_indices, failed_res_emb, failed_pairs = find_self_embeds(embeddings, tokenizer)
print(f"Number of bad embeddings = {len(fail_indices)}")
torch.cuda.empty_cache()
gc.collect()

100%|██████████| 13/13 [00:06<00:00,  1.90it/s]


Number of bad embeddings = 1037


In [7]:
nn = fail_indices[2]
nn = 2
self_emb = embeddings[nn]
X = torch.cat([embeddings[:nn], embeddings[nn+1:]], dim=0)
X = self_emb[None, :] - X;

In [8]:
X_zero = np.zeros((1, X.shape[1]))
# метки классов: вектора = +1, ноль = -1
X_all = np.vstack([X, X_zero])
y_all = np.hstack([np.ones(X.shape[0]), [-1]])

In [11]:
clf = LinearSVC(C=1e9, max_iter=1000, tol=1e-7)
clf.fit(X_all, y_all)
pred = clf.predict(X_all)

In [12]:
res = (pred != y_all)
error_ind = np.where(res)
n_errors = res.sum()
print(n_errors)

1
